In [1]:
import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
# complex valued tensor class
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from models import TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor
from preprocess import *

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, build_Theta, print_pde
from RegscorePy.bic import bic

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

N = 15000

DATA_PATH = '../experimental_data/NLS.mat'
data = io.loadmat(DATA_PATH)

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = to_column_vector(Exact_u)
v_star = to_column_vector(Exact_v)

idx = np.random.choice(X_star.shape[0], N, replace=False)

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

X_train = to_tensor(X_star[idx, :], True).to(device)
u_train = to_tensor(u_star[idx, :], False).to(device)
v_train = to_tensor(v_star[idx, :], False).to(device)

feature_names = ['hf', '|hf|', 'h_x', 'h_xx', 'h_xxx']

You're running on cpu


In [3]:
spatial_dim = x.shape[0]
time_dim = t.shape[0]

dt = (t[1]-t[0])[0]
dx = (x[2]-x[1])[0]

fd_h_t = np.zeros((spatial_dim, time_dim), dtype=np.complex64)
fd_h_x = np.zeros((spatial_dim, time_dim), dtype=np.complex64)
fd_h_xx = np.zeros((spatial_dim, time_dim), dtype=np.complex64)
fd_h_xxx = np.zeros((spatial_dim, time_dim), dtype=np.complex64)

for i in range(spatial_dim):
    fd_h_t[i,:] = FiniteDiff(Exact[i,:], dt, 1)
for i in range(time_dim):
    fd_h_x[:,i] = FiniteDiff(Exact[:,i], dx, 1)
    fd_h_xx[:,i] = FiniteDiff(Exact[:,i], dx, 2)
    fd_h_xxx[:,i] = FiniteDiff(Exact[:,i], dx, 3)
    
fd_h_t = np.reshape(fd_h_t, (spatial_dim*time_dim,1), order='F')
fd_h_x = np.reshape(fd_h_x, (spatial_dim*time_dim,1), order='F')
fd_h_xx = np.reshape(fd_h_xx, (spatial_dim*time_dim,1), order='F')
fd_h_xxx = np.reshape(fd_h_xxx, (spatial_dim*time_dim,1), order='F')

X_ders = np.hstack([np.ones((spatial_dim*time_dim,1)), fd_h_x, fd_h_xx, fd_h_xxx])
X_data = np.hstack([np.reshape(Exact, (spatial_dim*time_dim,1), order='F'), np.reshape(abs(Exact), (spatial_dim*time_dim,1), order='F')])
derivatives_description = ['','h_{x}','h_{xx}', 'h_{xxx}']

X, rhs_des = build_Theta(X_data, X_ders, derivatives_description, 3, data_description = ['h','|h|'])

w = TrainSTRidge(X, fd_h_t, 10**-5, 500)
print("PDE derived using STRidge")
print_pde(w, rhs_des)

PDE derived using STRidge
u_t = (-0.000001 +0.503178i)h_{xx}
    + (-0.000001 +1.000581i)h|h|^2
   
